In [337]:

import csv

from sympy import *

reader = csv.reader(open("transitions.csv", "r"))
csv_content = list(reader)
components = [component.lstrip() for component in csv_content[0]]
transition_matrix = csv_content[1:]
print(components)
print(len(transition_matrix))

['Authentication Service', 'Availability Item Filter', 'Bid and Buy Service', 'Buy Now Item Filter', 'Category Item Filter', 'Comment Item Filter', 'Future Sales Item Filter', 'Inventory Service', 'Item Management Service', 'Last Second Sales Item Filter', 'Past Sales Item Filter', 'Persistence Service', 'Query Service', 'Recommendation Item Filter', 'Region Item Filter', 'Reputation Service', 'Seller Reputation Item Filter', 'User Management Service', 'Supervisory Component']
19


In [338]:
'''
num_components = 19 # 18 components + supervisory component

# Inital component dependencies
def component_dependencies(i, j):
    if (i ==  0 and j == 12) or (i == 1 and j == 14) or (i == 3 and j == 1) or (i == 4 and j == 13) or (i == 5 and j == 4) or (i == 9 and j == 10) or (i == 10 and j == 3) or ((i == 11 or i == 6) and j == 18) or (i == 12 and j == 9) or (i == 13 and j == 6) or (i == 14 and j == 16) or (i == 16 and j == 5):
        return 1
    elif (i == 7 and (j == 11 or j == 12)):
        return 0.5
    elif ((i == 8 or i == 15 or i == 17) and j == 11):
        return 0.34
    elif ((i == 8 or i == 15 or i == 17) and (j == 0 or j == 12)):
        return 0.33
    elif (i == 2 and (j == 0 or j == 7 or j == 11 or j == 12)):
        return 0.25
    elif (i == 18 and j == 0):
        return 0.2
    elif (i == 18 and (j == 2 or j == 7 or j == 8 or j == 15 or j == 17)):
        return 0.16
    else:
        return 0
'''
P_c = Matrix(transition_matrix)  # Initial component transition probabilities
num_components = len(transition_matrix)

P_c

Matrix([
[   0,   0,    0,   0,   0,   0,   0,    0,    0,   0,   0,    0,  1.0,   0,   0,    0,   0,    0,   0],
[   0,   0,    0,   0,   0,   0,   0,    0,    0,   0,   0,    0,    0,   0, 1.0,    0,   0,    0,   0],
[0.25,   0,    0,   0,   0,   0,   0, 0.25,    0,   0,   0, 0.25, 0.25,   0,   0,    0,   0,    0,   0],
[   0, 1.0,    0,   0,   0,   0,   0,    0,    0,   0,   0,    0,    0,   0,   0,    0,   0,    0,   0],
[   0,   0,    0,   0,   0,   0,   0,    0,    0,   0,   0,    0,    0, 1.0,   0,    0,   0,    0,   0],
[   0,   0,    0,   0, 1.0,   0,   0,    0,    0,   0,   0,    0,    0,   0,   0,    0,   0,    0,   0],
[   0,   0,    0,   0,   0,   0,   0,    0,    0,   0,   0,    0,    0,   0,   0,    0,   0,    0, 1.0],
[   0,   0,    0,   0,   0,   0,   0,    0,    0,   0,   0,  0.5,  0.5,   0,   0,    0,   0,    0,   0],
[0.33,   0,    0,   0,   0,   0,   0,    0,    0,   0,   0, 0.34, 0.33,   0,   0,    0,   0,    0,   0],
[   0,   0,    0,   0,   0,   0,   0,    0,   

In [339]:
# Inner state of component: operational (0), degrading (1) or unresponsive (2)

# Mapping
states = ("operational", "degrading", "unresponsive")
operational = states.index("operational")
degrading = states.index("degrading")
unresponsive = states.index("unresponsive")


def inner_states(i, j):
    if i == j == operational:
        return 0.15
    elif i == j == degrading:
        return 0.25
    elif i == j == unresponsive:
        return 0.5
    elif i == operational and j == degrading:
        return 0.05
    elif i == degrading and j == operational:
        return 0.5
    elif i == degrading and j == unresponsive:
        return 0.1
    elif i == unresponsive and j == operational:
        return 0.25
    else:
        return 0


P_i = Matrix(3, 3, inner_states)

missing_outgoing = ones(P_i.shape[1], 1) - P_i * ones(P_i.shape[1], 1)

# Further transition probabilities depend if component is 
for i in range(num_components):
    if P_c[:, i] == zeros(num_components, 1):
        # a) only calling other components
        # o_component -> o_called, u_component -> u_called
        called_components = P_c[i, :]
        number_of_called_components = len([e for e in called_components if e != 0])
        print("Component {} is only calling other components ({} components)".format(i, number_of_called_components))
        distributed_outgoing = missing_outgoing / number_of_called_components
    elif P_c[i, :] == zeros(1, num_components):
        # b) only being called by other components
        # o_component -> o_calling, d_component -> d_calling, u_component -> u_calling
        print("Component {} is only called by other components".format(i))
    else:
        # c) calling other components and being called by other components
        # o_component -> o_called and o_calling, d_component -> d_called and d_calling, u_component -> u_called and u_calling
        print("Component {} is bidirectional".format(i))

P_i

Component 0 is bidirectional
Component 1 is bidirectional
Component 2 is bidirectional
Component 3 is bidirectional
Component 4 is bidirectional
Component 5 is bidirectional
Component 6 is bidirectional
Component 7 is bidirectional
Component 8 is bidirectional
Component 9 is bidirectional
Component 10 is bidirectional
Component 11 is bidirectional
Component 12 is bidirectional
Component 13 is bidirectional
Component 14 is bidirectional
Component 15 is bidirectional
Component 16 is bidirectional
Component 17 is bidirectional
Component 18 is bidirectional


Matrix([
[0.15, 0.05,   0],
[ 0.5, 0.25, 0.1],
[0.25,    0, 0.5]])

In [356]:
def initial_transition_probabilities(i, j):
    return 0


P_t = Matrix(P_c.shape[0] * P_i.shape[0], P_c.shape[1] * P_i.shape[1],
             initial_transition_probabilities)  # Transition probabilities including inner states

for i in range(P_c.shape[0]):
    # Initialize diagonal of P_t
    for j in range(P_c.shape[1]):
        if i == j:
            # inner states (no outgoing)
            for x in range(i * 3, (i * 3) + 3):
                for y in range(j * 3, (j * 3) + 3):
                    P_t[x, y] = P_i[x % 3, y % 3]
    row = P_c[i, :]
    col = P_c[:, i]
    # Check type of component
    if P_c[:, i] == zeros(num_components, 1):
        # Only calling other components
        prob_dict = {j: row[j] for j in range(len(row)) if row[j] != 0}
        for c in prob_dict:
            values = [prob_dict[c] * miss for miss in missing_outgoing]
            t_row_index_o = i * 3 + operational
            t_col_index_o = c * 3 + operational
            t_row_index_d = i * 3 + degrading
            t_col_index_d = c * 3 + degrading
            P_t[t_row_index_o, t_col_index_o] = values[operational]
            P_t[t_row_index_d, t_col_index_d] = values[degrading]
        # Unresponsive at inner unresponsive state
        P_t[i * 3 + unresponsive, i * 3 + unresponsive] += values[unresponsive]
    elif P_c[i, :] == zeros(1, num_components):
        # Only called by other components
        sum = ones(1, col.shape[0]) * col
        normalized_col = col / sum[0, 0]
        prob_dict = {j: normalized_col[j] for j in range(len(normalized_col)) if normalized_col[j] != 0}
        for c in prob_dict:
            values = [prob_dict[c] * miss for miss in missing_outgoing]
            t_row_index_o = i * 3 + operational
            t_col_index_o = c * 3 + operational
            t_row_index_d = i * 3 + degrading
            t_col_index_d = c * 3 + degrading
            t_row_index_u = i * 3 + unresponsive
            t_col_index_u = c * 3 + unresponsive
            P_t[t_row_index_o, t_col_index_o] = values[operational]
            P_t[t_row_index_d, t_col_index_d] = values[degrading]
            P_t[t_row_index_u, t_col_index_u] = values[unresponsive]
    else:
        # Bidirectional component
        # Rows
        prob_dict = {j: row[j] for j in range(len(row)) if row[j] != 0}
        for c in prob_dict:
            values = [prob_dict[c] * miss for miss in missing_outgoing * 0.5]
            t_row_index_o = i * 3 + operational
            t_col_index_o = c * 3 + operational
            t_row_index_d = i * 3 + degrading
            t_col_index_d = c * 3 + degrading
            t_row_index_u = i * 3 + unresponsive
            t_col_index_u = c * 3 + unresponsive
            P_t[t_row_index_o, t_col_index_o] = values[operational]
            P_t[t_row_index_d, t_col_index_d] = values[degrading]
            P_t[t_row_index_u, t_col_index_u] = values[unresponsive]
        # Columns
        sum = ones(1, col.shape[0]) * col
        normalized_col = col / sum[0, 0]
        prob_dict = {j: normalized_col[j] for j in range(len(normalized_col)) if normalized_col[j] != 0}
        for c in prob_dict:
            values = [prob_dict[c] * miss for miss in missing_outgoing * 0.5]
            t_row_index_o = i * 3 + operational
            t_col_index_o = c * 3 + operational
            t_row_index_d = i * 3 + degrading
            t_col_index_d = c * 3 + degrading
            t_row_index_u = i * 3 + unresponsive
            t_col_index_u = c * 3 + unresponsive
            P_t[t_row_index_o, t_col_index_o] = values[operational]
            P_t[t_row_index_d, t_col_index_d] = values[degrading]
            P_t[t_row_index_u, t_col_index_u] = values[unresponsive]

with open("data2.txt", "w") as file:
    file.write(str(P_t))

P_t

Matrix([
[             0.15,               0.05,                  0,    0,     0,     0, 0.0694444444444444,                   0,                  0,    0,     0,     0,    0,     0,     0,    0,     0,     0,    0,      0,      0,                 0,                  0,                  0, 0.0916666666666667,                   0,                  0,    0,     0,     0,    0,     0,     0,     0,       0,       0,   0.4,       0,       0,    0,     0,     0,    0,     0,     0, 0.0916666666666667,                   0,                  0,    0,     0,     0, 0.0916666666666667,                   0,                  0, 0.0555555555555556,                  0,                  0],
[              0.5,               0.25,                0.1,    0,     0,     0,                  0,  0.0130208333333333,                  0,    0,     0,     0,    0,     0,     0,    0,     0,     0,    0,      0,      0,                 0,                  0,                  0,                  0,           0.0

In [341]:
# Check transition matrix
row_sums = P_t * ones(P_t.shape[1], 1)
row_sums == ones(P_t.shape[0], 1)
P_t * ones(P_t.shape[1], 1)


Matrix([
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0]])

In [342]:
from collections import deque
# random walks starting at random component
import random

trace_len = 10
number_of_traces = 1000
raw_traces = []

for j in range(number_of_traces):
    # chose a random component from the available ones
    start_component = random.randint(0, 18) * 3
    start_vector = zeros(1, num_components * 3)
    start_vector[start_component] = 1
    trace = deque()
    trace.append(start_component)
    current_vector = start_vector
    for i in range(trace_len - 1):
        next_step_probability_distribution = current_vector * P_t
        next_index = random.choices(range(0, len(next_step_probability_distribution)),
                                    weights=next_step_probability_distribution, k=1)[0]
        trace.append(next_index)
        current_vector = zeros(1, num_components * 3)
        current_vector[next_index] = 1

    raw_traces.append(trace)


In [343]:
from collections import namedtuple

TraceItem = namedtuple("TraceItem", "component state")

traces = [deque(TraceItem(components[item // 3], states[item % 3]) for item in raw_trace) for raw_trace in raw_traces]


In [344]:



def check_normal_operation(inner_trace: deque) -> bool:
    return all(item.state == "operational" for item in inner_trace)


def check_failure_masking(inner_trace: deque) -> bool:
    for (index, value) in enumerate(inner_trace):
        if value.state == "unresponsive" and index <= len(inner_trace) - 3:
            if inner_trace[index + 1].component != value.component and inner_trace[index + 1].state == "unresponsive":
                if inner_trace[index + 2].component == value.component and inner_trace[
                    index + 2].state == "operational":
                    return True
    return False


def check_systemic_degradation(inner_trace: deque) -> bool:
    for (index, value) in enumerate(inner_trace):
        if value.state == "degrading" and index < len(inner_trace) - 1:
            if inner_trace[index + 1].component != value.component and inner_trace[index + 1].state == "degrading":
                return True
    return False


# failure_mask_injection = deque([TraceItem(component='Bid and Buy Service', state='operational'), TraceItem(component='Supervisory Component', state='operational'), TraceItem(component='Authentication Service', state='operational'), TraceItem(component='Item Management Service', state='operational'), TraceItem(component='Item Management Service', state='degrading'), TraceItem(component='Item Management Service', state='unresponsive'), TraceItem(component='Item Management Service', state='unresponsive'), TraceItem(component='Item Management Service', state='unresponsive'), TraceItem(component='Authentication Service', state='unresponsive'), TraceItem(component='Item Management Service', state='operational')])
# traces.append(failure_mask_injection)

traces_operational = [check_normal_operation(inner_trace) for inner_trace in traces]
traces_failure_masked = [check_failure_masking(inner_trace) for inner_trace in traces]
traces_systemic_degradation = [check_systemic_degradation(inner_trace) for inner_trace in traces]
print("Operational Traces", traces_operational.count(True))
print("Failure Masked Traces", traces_failure_masked.count(True))
print("Systemic Degredation Traces", traces_systemic_degradation.count(True))

# for i, b in ((i, a) for i, a in enumerate(traces) if any(item.state == "unresponsive" for item in a)):
#     print(i, [value.state for value in b])


Operational Traces 631
Failure Masked Traces 0
Systemic Degredation Traces 75
40 ['operational', 'operational', 'operational', 'operational', 'operational', 'degrading', 'degrading', 'unresponsive', 'operational', 'operational']
50 ['operational', 'operational', 'operational', 'operational', 'operational', 'operational', 'operational', 'degrading', 'degrading', 'unresponsive']
69 ['operational', 'operational', 'operational', 'operational', 'operational', 'operational', 'operational', 'operational', 'degrading', 'unresponsive']
72 ['operational', 'operational', 'operational', 'operational', 'operational', 'degrading', 'unresponsive', 'unresponsive', 'unresponsive', 'unresponsive']
75 ['operational', 'operational', 'operational', 'operational', 'operational', 'operational', 'degrading', 'unresponsive', 'unresponsive', 'unresponsive']
152 ['operational', 'degrading', 'operational', 'operational', 'operational', 'degrading', 'unresponsive', 'operational', 'operational', 'operational']
161 

In [24]:
import csv

from sympy import Matrix, eye, ones
from sympy.physics.quantum import TensorProduct

reader = csv.reader(open("transitions.csv", "r"))
csv_content = list(reader)
components = [component.lstrip() for component in csv_content[0]]
transition_matrix = csv_content[1:]

component_transitions = Matrix(transition_matrix)  # Initial component transition probabilities
num_components = len(transition_matrix)

states = ("operational", "degrading", "unresponsive")
inner_state_transitions = Matrix([[0.15, 0.05, 0.00],
                                  [0.50, 0.25, 0.10],
                                  [0.25, 0.00, 0.50]])

self_loops = TensorProduct(eye(num_components), inner_state_transitions)
inverse_self_loop_sums = self_loops * ones(self_loops.shape[1], 1)
inverse_self_loop_sums = inverse_self_loop_sums.applyfunc(lambda a: 1 - a)

transitions = TensorProduct(component_transitions, eye(len(states)))

failure_masking_identity = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
transitions_failure_masking = TensorProduct(component_transitions.T, failure_masking_identity)

full_transitions_without_self_loops = transitions + transitions_failure_masking
full_transitions_without_self_loops_sums = full_transitions_without_self_loops * ones(
    full_transitions_without_self_loops.shape[1], 1)
full_transitions_without_self_loops_sums = full_transitions_without_self_loops_sums.applyfunc(lambda a: a ** -1)

normalizing_vector = inverse_self_loop_sums.multiply_elementwise(full_transitions_without_self_loops_sums)
normalizing_matrix = normalizing_vector * ones(1, full_transitions_without_self_loops.shape[0])

normalized_transition_matrix = full_transitions_without_self_loops.multiply_elementwise(normalizing_matrix)
final_transition_matrix = normalized_transition_matrix + self_loops

final_transition_matrix

Matrix([
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0],
[1.0]])

In [8]:
self_loops

Matrix([
[0.15, 0.05,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0],
[ 0.5, 0.25, 0.1,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0],
[0.25,    0, 0.5,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0,    0,    0,   0],
[   0,    0,   0